In [17]:
import sys
print(sys.executable)


c:\Users\adhik\OneDrive\Desktop\job_recommendation_system\venv\Scripts\python.exe


In [18]:
import os
import numpy as np
import pandas as pd


In [19]:
df = pd.read_csv("../backend/models/jobs_with_features.csv")
print("Loaded:", df.shape)
df.head(3)


Loaded: (986, 18)


,jobid,job_title,description_text,benefits,job_type,location,salary_formatted,company_name,company_rating,company_reviews_count,country,url,apply_link,job_title_lc,company_name_lc,location_lc,combined_text,description_short
0,d46480488e3aa20c,FitLab Email Marketing Manager,Description: \n Who Is FitLab: FitLab is a p...,"[""401(k)"",""Dental insurance"",""Disability insur...",Full-time,California,"$90,000 - $105,000 a year",FitLab INC,4.5,4,US,https://www.indeed.com/viewjob?jk=d46480488e3a...,https://www.indeed.com/viewjob?jk=d46480488e3a...,fitlab email marketing manager,fitlab inc,california,fitlab email marketing manager description who...,Description: \n Who Is FitLab: FitLab is a p...
1,ba9bf4e04cfd9cdb,Customer Service Representative - Work From Home,The Customer Associate is the first point of c...,"[""401(k) matching"",""Continuing education credi...",Permanent,"Sugar Land, TX 77479",$28 - $50 an hour,Buccal Up Dental,5.0,4,US,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,customer service representative - work from home,buccal up dental,"sugar land, tx 77479",customer service representative work from home...,The Customer Associate is the first point of c...
2,9e4031d06dd42d32,Information Technology Support Specialist,About Peraton Peraton is a next-generation na...,"[""Dental insurance"",""Disability insurance"",""He...",Full-time,United States,"$66,000 - $106,000 a year",Peraton,3.1,297,US,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,information technology support specialist,peraton,united states,information technology support specialist abou...,About Peraton Peraton is a next-generation na...


In [20]:
assert "combined_text" in df.columns, "combined_text missing; run Notebook 02 again."


In [21]:
from sentence_transformers import SentenceTransformer


In [22]:
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)
print("Loaded model:", model_name)


Loaded model: all-MiniLM-L6-v2


In [23]:
os.makedirs("../backend/models", exist_ok=True)
emb_path = "../backend/models/sbert_job_embeddings.npy"


In [24]:
if os.path.exists(emb_path):
    job_embeddings = np.load(emb_path)
    print("Loaded cached embeddings:", job_embeddings.shape)
else:
    texts = df["combined_text"].astype(str).tolist()
    job_embeddings = model.encode(
        texts,
        batch_size=32,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    np.save(emb_path, job_embeddings)
    print("Computed + saved embeddings:", job_embeddings.shape)


Loaded cached embeddings: (986, 384)


In [25]:
def recommend_sbert(
    profile_text: str,
    top_n: int = 10,
    location: str | None = None,
    job_type: str | None = None
) -> pd.DataFrame:

    profile_text = str(profile_text).strip()
    if not profile_text:
        raise ValueError("profile_text is empty.")

    q_emb = model.encode(
        [profile_text],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0]  # shape: (d,)

    # cosine similarity for normalized vectors = dot product
    sims = job_embeddings @ q_emb  # shape: (num_jobs,)

    results = df.copy()
    results["score"] = sims

    # Optional filters
    if location and location.lower() != "any":
        results = results[results["location"].str.contains(location, case=False, na=False)]

    if job_type and job_type.lower() != "any":
        results = results[results["job_type"].str.contains(job_type, case=False, na=False)]

    results = results.sort_values("score", ascending=False).head(top_n)

    show_cols = [c for c in [
        "job_title", "company_name", "location", "job_type", "salary_formatted", "company_rating", "score", "url", "apply_link"
    ] if c in results.columns]

    return results[show_cols]


In [26]:
recommend_sbert("frontend developer react internship", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
356,Lead Fullstack Engineer / Java / React / WFH F...,Motion Recruitment,"Lake Forest, IL 60045",Full-time,"$150,000 - $170,000 a year",3.8,0.533790,https://www.indeed.com/viewjob?jk=4d599b0755f1...,https://www.indeed.com/viewjob?jk=4d599b0755f1...
428,Risk Engineering Training Program - Colorado -...,Zurich Insurance Company Ltd.,"Denver, CO",Full-time,"$75,400 a year",3.8,0.343808,https://www.indeed.com/viewjob?jk=e6c976804bf4...,https://www.indeed.com/viewjob?jk=e6c976804bf4...
969,"Sr Manager, Product Management",Thermo Fisher Scientific,"South San Francisco, CA 94080",Full-time,"$143,000 - $195,000 a year",3.4,0.323684,https://www.indeed.com/viewjob?jk=f563c260fcd7...,https://www.indeed.com/viewjob?jk=f563c260fcd7...
470,AI Training for Mathematics,Outlier,Remote,"Contract, Freelance",Up to $17 an hour,1.9,0.315797,https://www.indeed.com/viewjob?jk=188d81d64bbc...,https://www.indeed.com/viewjob?jk=188d81d64bbc...
265,Senior Applications Developer,Buildertrend,"Omaha, NE 68137",Full-time,"$115,000 - $165,000 a year",3.6,0.313184,https://www.indeed.com/viewjob?jk=c269f320bff3...,https://www.indeed.com/viewjob?jk=c269f320bff3...


In [27]:
recommend_sbert("data analyst python sql excel", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
631,Data Analyst,DAP Health,"Palm Springs, CA 92262",Full-time,$32.49 - $33.97 an hour,3.1,0.536408,https://www.indeed.com/viewjob?jk=0415f5a87724...,https://www.indeed.com/viewjob?jk=0415f5a87724...
220,Healthcare Analyst,Healthfirst,"100 Church St, New York, NY 10007",Full-time,"$67,200 - $97,815 a year",3.3,0.434034,https://www.indeed.com/viewjob?jk=f8e5ccf9d3c3...,https://www.indeed.com/viewjob?jk=f8e5ccf9d3c3...
527,Data Analyst (Remote),Medwatch LLC,Remote,Full-time,"$80,000 - $90,000 a year",3.3,0.427983,https://www.indeed.com/viewjob?jk=5a0e36d66b9f...,https://www.indeed.com/viewjob?jk=5a0e36d66b9f...
833,Senior Healthcare Analyst,VNS Health,"Manhattan, NY",Full-time,"$93,400 - $116,800 a year",3.6,0.413159,https://www.indeed.com/viewjob?jk=90c8253adcf6...,https://www.indeed.com/viewjob?jk=90c8253adcf6...
807,Data Analyst - Immigration Data,Fragomen,"New York, NY",Full-time,"$69,000 - $78,000 a year",3.3,0.398322,https://www.indeed.com/viewjob?jk=3d5e7c1c3550...,https://www.indeed.com/viewjob?jk=3d5e7c1c3550...


In [28]:
recommend_sbert("customer service work from home", top_n=5, location="Remote")


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
627,French Canadian Customer Support,Bread Financial,Remote,Full-time,$18.55 - $34.00 an hour,4.1,0.496712,https://www.indeed.com/viewjob?jk=45dad4c64662...,https://www.indeed.com/viewjob?jk=45dad4c64662...
662,Client Services Representative,North American Bancard,Remote,Full-time,$14 - $21 an hour,3.2,0.496161,https://www.indeed.com/viewjob?jk=072984f991f2...,https://www.indeed.com/viewjob?jk=072984f991f2...
87,Operator - Contractors/Home Services,Answering Service Care,Remote,Full-time,$15 - $17 an hour,3.8,0.466628,https://www.indeed.com/viewjob?jk=1b1e3642b0a1...,https://www.indeed.com/viewjob?jk=1b1e3642b0a1...
17,Customer Support Representative I,BSH Home Appliances Corporation,Remote,Full-time,$14.81 - $19.75 an hour,3.8,0.441944,https://www.indeed.com/viewjob?jk=765f24979ec9...,https://www.indeed.com/viewjob?jk=765f24979ec9...
256,Sales Support Representative,Safe Step Walk-In Tub,Remote,Full-time,$18.71 - $29.92 an hour,3.7,0.407117,https://www.indeed.com/viewjob?jk=12b674862862...,https://www.indeed.com/viewjob?jk=12b674862862...


In [29]:
recommend_sbert("software developer entry level", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,company_rating,score,url,apply_link
866,Software Engineer,Kforce,"San Francisco, CA 94105",Contract,$45 - $50 an hour,3.8,0.484863,https://www.indeed.com/viewjob?jk=7181d48cb651...,https://www.indeed.com/viewjob?jk=7181d48cb651...
585,Java Developer Level 2,Kforce,"Cincinnati, OH 45202",Contract,$67 - $72 an hour,3.8,0.475788,https://www.indeed.com/viewjob?jk=c328667a127a...,https://www.indeed.com/viewjob?jk=c328667a127a...
624,Senior Software Engineer II,PrintMail Solutions,Remote,Full-time,"$130,000 a year",2.9,0.468470,https://www.indeed.com/viewjob?jk=49cfe129ca93...,https://www.indeed.com/viewjob?jk=49cfe129ca93...
507,"Senior Product Manager, Technical, AWS Develop...","Amazon Web Services, Inc.","San Francisco, CA",Full-time,"$137,000 - $236,800 a year",3.4,0.467031,https://www.indeed.com/viewjob?jk=f17d8cad571c...,https://www.indeed.com/viewjob?jk=f17d8cad571c...
408,"Senior Software Developer PHP - Remote (CA, WA...",Providence Health Plan,"Salem, OR 97302",Full-time,$44.12 - $71.03 an hour,3.6,0.453360,https://www.indeed.com/viewjob?jk=2c599337296b...,https://www.indeed.com/viewjob?jk=2c599337296b...


In [30]:
import re

def keyword_overlap(query: str, text: str, top_k=8):
    q_tokens = set(re.findall(r"[a-zA-Z0-9]+", query.lower()))
    t_tokens = set(re.findall(r"[a-zA-Z0-9]+", text.lower()))
    overlap = list(q_tokens.intersection(t_tokens))
    return overlap[:top_k]


In [31]:
res = recommend_sbert("software developer entry level", top_n=3)
top_title = res.iloc[0]["job_title"]

# locate that row in df (approx match)
idx = df.index[df["job_title"] == top_title][0]
keyword_overlap("software developer entry level", df.loc[idx, "combined_text"])


['developer', 'software']

In [32]:
with open("../backend/models/sbert_model_name.txt", "w", encoding="utf-8") as f:
    f.write(model_name)

print("Saved SBERT model name:", model_name)


Saved SBERT model name: all-MiniLM-L6-v2
